## Radical Stabilisation Energy Calculation Workflow

This workflow takes a user through the entire process to the calculate the Radical Stabilisation Energy in a ligand-protein complex. It will hold you hand (if you want) throughout the entire workflow. If you would like to do something a little you will need to alter certain parts but hopefully this is clear!

It takes a PDB as starting structure parameterises, minimises and equilibrates the structure before running a production run. It then takes the ligand adds point charges and runs QM energy calculations. Finally it calculates the vRSE and produces some nice graphs. Ill work on a nice workflow diagram and paragraph to explain it as well.

Chris Suess 

christian.suess1@nottingham.ac.uk

In [ ]:
#This part checks for any dependencies we will need throughout the workflow

all_good = True

try:
    import numpy as np
except ImportError:
    print('Error - you neeed to install numpy')
    all_good = False
    
try:
    import mdtraj as mdt
except ImportError:
    print('Error - you need to install mdtraj')
    all_good = False

try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except ImportError:
    print('Error - you need to install matplotlib')
    all_good = False
    
try:
    import nglview as nv
except ImportError:
    print('Error - you need to install nglview')
    all_good = False
    
try:
    import pinda
except ImportError:
    print('Error - you need to install pinda')
    all_good = False

        
if all_good:
    print('Success - you seem to have everything ready to go.')

In [ ]:
from utilities import *
%matplotlib inline

pdb_code = '4nji' # downloads PDB from https://www.rcsb.org/ it can be changed for a local copy
structures = download(pdb_code)

In the next cell, we use the display() function to view our molecule:

In [ ]:
view0 = display(structures)
view0

In [ ]:
peptides = select(structures, 'protein') # only keep parts of the structure that are 'protein'
clean_structure = clean_up(peptides[model_number]) # cleans structure using pdb4amber

The below cell parameterises the protein-ligand complex using 'standard' methods

In [ ]:
parameters = ['leaprc.protein.ff14SB', 'leaprc.water.tip3p']
script = """

solvateoct x TIP3PBOX 10.0
addions x Cl- 0

"""
topology, start_coordinates = run_leap(parameters, script, clean_structure)

In [ ]:
view = display(start_coordinates, topology)
view

Next we run the energy minimisation job. The commands for this in the input script are very simple: a) do a minimization (imin=1), b) run for a maximum of 1000 steps (maxcyc=1000), c) use *periodic boundary conditions* (ntb=1). 

The minimize() command produces two outputs: the minimized coordinates, and a log file with information about what happened during rhe run. This is a fairly heavy calculation, the cell may take a minute to run.

In [ ]:
minimization_script = '''

&cntrl
 imin=1, maxcyc=1000,
 ntb=1,
 /
 
 '''

minimized_coordinates, logfile = minimize(minimization_script, start_coordinates, topology)
minimized_coordinates.save("em_out.ncrst")
logfile.save("em_out.log")

Look at the results. Load both the starting and final coordinates into the viewer, so you can see how the structure has changed over the minimization.

In [ ]:
view2 = display([start_coordinates, minimized_coordinates], topology)
view2

Before moving on, see how complete the enetgy minimisation process was, by plotting how the total energy of the system changed over the 1000 steps:

In [ ]:
plot_energies(logfile)

You should see that the energy minimisation seems to have been fairly succesful - the energy of the system is almost stable by the end of the run.

Now we run the main molecular dynamics (MD) simulation. The script is more detailed, you will need to consult the [Amber manual](http://ambermd.org/doc12/Amber16.pdf) to get the details. 

The run_md() command produces three outputs: the final coordinates of the system, a *trajectory* file, and another log file. This is a major computation, and will take a couple of minutes, or maybe longer, to complete.

In [ ]:
md_script = '''

&cntrl
 imin=0, irest=0, ntx=1, dt=0.002,
 ntt=3, temp0=300, gamma_ln=5.0,
 ntp=1, taup=2.0,
 ntb=2, ntc=2, ntf=2,
 nstlim=10000, ntwx=500, ntpr=500,
 /
 
 '''

final_coordinates, trajectory, md_log = run_md(md_script, minimized_coordinates, topology)
final_coordinates.save("md_out.ncrst")
trajectory.save("md_out.nc")
md_log.save("md_out.log")

Look at the results. The script above set the total simulation time to be twenty picoseconds, and asked for a snapshot to be saved to the trajectory every picosecond, so there are twenty frames in the animation.

In [ ]:
view3 = display(trajectory, topology)
view3

## vRSE Calculation using point charges

Extracts ligand from trajectory of MD simulations. Creates inputs for Single Point Energy Calculations.

Assumes a QM package is available

In [ ]:
view0 = display(structures)
view0

In [ ]:
ligand = select(structures, 'residue 2K8')

In [ ]:
view1 = display(ligand)
view1

## vRSE Calculation